## Downloading Files from Google Drive to HydroShare

Once exported a file from Google Earth Engine, we can bring it to HydroShare for further analysis or to set it up as a resource.

Do not forget to install the following the libraries in a terminal:

In [5]:
from __future__ import print_function
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
import io
from googleapiclient.http import MediaIoBaseDownload

We need the token.json and credentials.json files in the folder of this notebook

In [6]:
# verifying the Google Drive API is up and running
SCOPES = 'https://www.googleapis.com/auth/drive'

store = file.Storage('token.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
    creds = tools.run_flow(flow, store)
service = build('drive', 'v3', http=creds.authorize(Http()))

We need the name of the file exported from Earth Engine. In this case LC08_123032_20140515_TOA.tif

In [7]:
filename='LC08_123032_20140515_TOA.tif' #here we identify the file of interest
query='\"name contains \''+filename+'\'\"'
print(query)

"name contains 'LC08_123032_20140515_TOA.tif'"


In [8]:
# Call the Drive v3 API
results = service.files().list(q="name contains 'LC08_123032_20140515_TOA.tif'",pageSize=100, fields="nextPageToken, files(name, id)").execute()
# results = service.files().list(q=print(query),pageSize=100, fields="nextPageToken, files(name, id)").execute()

items = results.get('files', [])

import pandas as pd
df =pd.DataFrame.from_dict(items)
print(df)
id = df.iloc[0]['id']
# print(id)

                                  id                          name
0  16YYnFWfyat9wYeX4lG51TjtkTdLcocBX  LC08_123032_20140515_TOA.tif
1  1hw_ShbBSks8pwR7G5WS8UBoPXVauMjMw  LC08_123032_20140515_TOA.tif


The previous cell should give an id like this:1xogAczbNGN8Em_WH6NH8v5GiRSuFMfnV_-SX5oJyxww
This is an internal id that indicates that the file is in fact within Google Drive

For GoogleDrive API, we need the id of the file to download to HydroShare

In [9]:
file_id = id
request = service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print ("Download %d%%." % int(status.progress() * 100))

Download 100%.


The file is downloaded to memory. It needs to be written to a file.

In [10]:
#writting the file from memory to local storage
fh.seek(0)

with open(df.iloc[0]['name'], "wb") as wer:
	wer.write(fh.read())

That's it! The file is available for any other calculation in HS

In [11]:
ls

0 Git commands for HydroShare.ipynb*
1 Google Earth Engine installation in HydroShare.ipynb*
2 Using Earth Engine in HydroShare.ipynb*
asset_upload/
credentials.json
Google Drive installation in HydroShare.ipynb
Lab 0 Getting to know EarthEngine (Part 1).ipynb*
Lab 0 Getting to know EarthEngine (Part 2).ipynb*
Lab 0 Getting to know EarthEngine (Part 3).ipynb*
Lab 0 Getting to know EarthEngine (Part 4).ipynb
Lab 1 Intro to Remote Sensing and EE.ipynb*
Lab 2 Characteristics of remotely sensed data.ipynb
LC08_123032_20140515_TOA.tif
leaflet_examples_tested_w_hydroshare/
LICENSE
quickstart.py
README.md*
token.json


How, we can use the file in Jupyter. See this example:

In [12]:
import rasterio
dataset = rasterio.open(filename)

In [13]:
dataset.name

'LC08_123032_20140515_TOA.tif'

In [14]:
dataset.count

3

In [15]:
from rasterio.plot import show
show(dataset)

<Figure size 640x480 with 1 Axes>

In [16]:
from osgeo import gdal, osr

gdal.UseExceptions()


fname = dataset.name

ds = gdal.Open(fname)
data = ds.ReadAsArray()
gt = ds.GetGeoTransform()
proj = ds.GetProjection()

inproj = osr.SpatialReference()
inproj.ImportFromWkt(proj)

print(inproj)

PROJCS["WGS 84 / UTM zone 50N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",117],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32650"]]


In [17]:
import cartopy.crs as ccrs


projcs = inproj.GetAuthorityCode('PROJCS')
projection = ccrs.epsg(projcs)
print(projection)

ParseError: mismatched tag: line 33, column 4 (<string>)

In [ ]:
import matplotlib.pyplot as plt

subplot_kw = dict(projection=projection)
fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=subplot_kw)

extent = (gt[0], gt[0] + ds.RasterXSize * gt[1],
          gt[3] + ds.RasterYSize * gt[5], gt[3])

img = ax.imshow(data[1:3,:,:], extent=extent,
                origin='upper')

<a id='section1'></a>
### 1. How to connect with HydroShare 
To interact with HydroShare, import the `hydroshare` utility library so that you can query and download resources into the JupyterHub environment.  You will be prompted for your HydroShare login credentials, which are necessary to establish a secure connection with your account.  
Execute the following cell by pressing `Shift-Enter`, or by pressing the play button 
<img style='display:inline;padding-bottom:15px' src='help/static/play-button.png'>
on the toolbar above.

In [ ]:
import os
from utilities import hydroshare
hs = hydroshare.hydroshare()

<a id='section4'></a>
### 4. Creating a new HydroShare resource

The best way to save your data is to put it back into HydroShare and is done using the `createHydroShareResource` function. The first step is to identify the files you want to save to a HydroShare.  The cell below lists all the files in the current working directory.

In [ ]:
# Grab all the files in the folder where you are working  
files = !ls
print('My files: %s' % files)

In [ ]:
!echo "This is my new file" > myfile.txt

In [ ]:
# lets save this content as a new resource in HydroShare
abstract = 'This is a demo of the HydroShare Python Notebook Server and Gdrive'
title = 'Hello From JupyterHub and Gdrive'    
keywords = ['Demo', 'JupyterHub', 'Gdrive']  
rtype = 'genericresource'          

# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title, 
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=['sebbop.tif', 'myfile.txt'], 
                                          public=False)